# Разработка собственных моделей

Разработка новых моделей состоит из следующих шагов:

1. Выбор подходящего базового класса
2. Реализация необходимых методов и добавление необходимых атрибутов

В простейшем варианте достаточно реализовать метод `score_hrt`, который принимает на вход батч идентификаторов сущностей и отношений и возвращает оценку для каждой тройки. 

Для получения эмбеддингов можно использовать атрибуты `entity_representations` и `relation_representations`, которые для моделей типа DistMult представляют собой список из 1 объекта класса `pykeen.nn.Embedding`. 

In [13]:
from pykeen.models.base import EntityRelationEmbeddingModel
from pykeen.nn.init import xavier_uniform_, xavier_normal_norm_
from torch.nn import functional
from pykeen.nn.emb import EmbeddingSpecification

class MyModel(EntityRelationEmbeddingModel):
    def __init__(self, embedding_dim: int = 50, **kwargs):
        # код из туториала на текущей версии не работает
        # приходится явно определять EmbeddingSpecification
        entity_repr = EmbeddingSpecification(embedding_dim=50, 
                                             initializer=xavier_uniform_, 
                                             constrainer=functional.normalize)
        relation_repr = EmbeddingSpecification(embedding_dim=50, 
                                               initializer=xavier_normal_norm_)
        super().__init__(entity_representations=entity_repr, 
                         relation_representations=relation_repr,
                         **kwargs)

    def score_hrt(self, hrt_batch):
        h = self.entity_representations[0](hrt_batch[:, 0])
        r = self.relation_representations[0](hrt_batch[:, 1])
        t = self.entity_representations[0](hrt_batch[:, 2])
        # по смыслу score - это число; в туториале отсутствует сумма
        # в реализации DistMult сумма есть
        return (h * r.sigmoid() * t).sum(dim=-1).view(-1, 1)


In [14]:
from pykeen.pipeline import pipeline

res = pipeline(
    model=MyModel,
    dataset='Nations',
    loss='NSSA',
)

Training epochs on cuda: 100%|██████████| 5/5 [00:00<00:00,  7.60epoch/s, loss=0.000117, prev_loss=0.000118]
INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=201.
Evaluating on cuda:   0%|          | 0.00/201 [00:00<?, ?triple/s]WARNING:pykeen.models.base:Calculations will fall back to using the score_hrt method, since this model does not have a specific score_h function. This might cause the calculations to take longer than necessary.
Evaluating on cuda: 100%|██████████| 201/201 [00:00<00:00, 8.38ktriple/s]
INFO:pykeen.evaluation.evaluator:Evaluation took 0.03s seconds
